# 第一層過濾可能的文章組合

如果文章內提到相同的作物則先歸類為一區，再在內部進行比較蟲害及疫病的重疊程度

In [3]:
import pandas as pd
import numpy as np

In [8]:
# 將作物名稱訂為一個 dictionary，並對同義異字的字詞對應到相同的數字
file = open("train/KeyWords/02crop_list.csv", "r", encoding='utf-8-sig')
rows = file.readlines()
crop_dict, crop_list = {}, []

count = 0
for row in rows:
    items = row.strip("\n").split(",")
    items = [item for item in items if (len(item) > 0 and item != " ")]  # 去掉多餘元素
    for item in items:
        crop_dict[item] = count
    crop_list.append(items[0])
    count += 1
print(crop_dict)

{'文旦柚': 0, '文旦': 0, '麻豆文旦': 0, '水稻': 1, '青蔥': 2, '蔥': 2, '龍鬚菜': 3, '韭菜': 4, '甘藍': 5, '西瓜': 6, '芋頭': 7, '香蕉': 8, '蕉農': 8, '蕉園': 8, '蕉株': 8, '甘藷': 9, '玉米': 10, '胡瓜': 11, '絲瓜': 12, '洋香瓜': 13, '菠菜': 14, '萵苣': 15, '十字花科': 16, '豆科': 17, '大蒜': 18, '茼蒿': 19, '落花生': 20, '花生': 20, '蔬菜': 21, '田菁': 22, '毛豆': 23, '豌豆': 24, '蓮花': 25, '梨': 26, '中國梨': 26, '高接梨': 26, '荔枝': 27, '玉荷包荔枝': 27, '龍眼': 28, '蓮霧': 29, '欒樹': 30, '台灣欒樹': 30, '果樹': 31, '草莓': 32, '草莓苗期': 32, '柑橘': 33, '紅棗': 34, '柿子': 35, '甜柿': 35, '桃子': 36, '桃': 36, '李子': 37, '枇杷': 38, '楊桃': 39, '檬果': 40, '芒果': 40, '木瓜': 41, '番石榴': 42, '芭樂': 42, '檸檬': 43, '印度棗': 44, '番茄': 45, '小果番茄': 45, '橙蜜香番茄': 45, '橙果小番茄': 45, '馬鈴薯': 46, '紅龍果': 47, '棗': 48, '蜜棗': 49, '瓜果類': 50, '豆菜類': 51, '洋蔥': 52, '豆類': 53, '龍柏': 54, '棗子': 55, '高粱': 56, '大豆': 57, '香瓜': 58, '南瓜': 59, '橄欖': 60, '葡萄': 61, '洋桔梗': 62, '柑桔': 63, '百香果': 64, '番荔枝': 65, '鳳梨釋迦': 66, '釋迦': 66, '樹豆': 67, '桶柑': 68, '茂谷柑': 69, '臍橙': 70, '晚崙西亞': 71, '晚崙西亞橙': 71, '丁香': 71, '荖花': 72, '荖葉': 73, '水蜜桃': 74, '小米': 

In [14]:
currentFolder = "train"

### 聚合同一類作物文章
創造一個 DataFrame，將相同文章的文章編號記錄下來

In [6]:
def put_to_df(fileNum):
    file = open(f"{currentFolder}/data/{fileNum}.csv", "r")
    rows = file.readlines()

    # extract crop info
    crop_row = rows[0].strip("\n").split(",")
    # too many crops, ignore it
    if(len(crop_row) > 5):
        if("香蕉" in crop_row):
            crop_row = ["香蕉"]
        elif("水稻" in crop_row):
            crop_row = ["水稻"]

    for item in crop_row:
        if(item == ''):
            continue
        index = crop_dict[item]
        if(int(fileNum) not in df.iloc[index]["articles"]):
            df.iloc[index]["articles"].append(int(fileNum))

In [15]:
df = pd.read_csv("train/KeyWords/02crop_list.csv", encoding='utf-8-sig', header=None)
df["articles"] = pd.Series([[] for i in range(len(df))])
df.head(10)

,0,1,2,3,articles
0,文旦柚,文旦,麻豆文旦,NaN,[]
1,水稻,NaN,NaN,NaN,[]
2,青蔥,蔥,NaN,NaN,[]
3,龍鬚菜,NaN,NaN,NaN,[]
4,韭菜,NaN,NaN,NaN,[]
5,甘藍,NaN,NaN,NaN,[]
6,西瓜,NaN,NaN,NaN,[]
7,芋頭,NaN,NaN,NaN,[]
8,香蕉,蕉農,蕉園,蕉株,[]
9,甘藷,NaN,NaN,NaN,[]


In [17]:
# scan 每個 file 並放到 df 當中
from os import listdir
from os.path import isfile, isdir, join

path = f"{currentFolder}/data"
files = listdir(path)

for f in files:
    fullpath = join(path, f)
    if isfile(fullpath):
        if(".csv" not in fullpath):
            continue
        fileNum = f.split(".")[0]
        put_to_df(fileNum)
print("finish", len(files))

finish 561


In [22]:
df.to_csv(f"{currentFolder}/crop_df.csv", index=False)
df.head(10)

,0,1,2,3,articles
0,文旦柚,文旦,麻豆文旦,NaN,"[747, 381, 351, 387, 1, 783, 636, 780, 733, 68..."
1,水稻,NaN,NaN,NaN,"[223, 237, 586, 790, 141, 97, 83, 54, 40, 800,..."
2,青蔥,蔥,NaN,NaN,"[544, 741, 620, 797, 3, 874, 726, 1073, 864, 6..."
3,龍鬚菜,NaN,NaN,NaN,[]
4,韭菜,NaN,NaN,NaN,"[885, 968, 824]"
5,甘藍,NaN,NaN,NaN,"[784, 862, 876]"
6,西瓜,NaN,NaN,NaN,"[755, 973, 262, 879, 107]"
7,芋頭,NaN,NaN,NaN,[81]
8,香蕉,蕉農,蕉園,蕉株,"[1199, 1172, 1173, 1198, 1239, 1207, 1170, 120..."
9,甘藷,NaN,NaN,NaN,"[741, 1073, 689, 649, 189, 415]"


In [23]:
# 作物文章統計
for i in range(len(df)):
    count = len(df.iloc[i]["articles"])
    if(count > 0):
        print(df.iloc[i][0], count)

文旦柚 21
水稻 233
青蔥 17
韭菜 3
甘藍 3
西瓜 5
芋頭 1
香蕉 79
甘藷 6
玉米 12
胡瓜 6
絲瓜 4
洋香瓜 11
菠菜 3
萵苣 3
十字花科 8
豆科 11
大蒜 6
茼蒿 5
落花生 10
蔬菜 20
田菁 6
毛豆 6
豌豆 5
蓮花 13
梨 10
荔枝 27
龍眼 18
蓮霧 11
欒樹 2
果樹 54
草莓 6
柑橘 15
紅棗 2
柿子 6
桃子 3
李子 1
枇杷 6
楊桃 15
檬果 38
木瓜 9
番石榴 17
印度棗 1
番茄 25
馬鈴薯 10
紅龍果 1
棗 4
瓜果類 2
豆菜類 1
洋蔥 3
龍柏 5
高粱 1
大豆 1
香瓜 1
南瓜 2
橄欖 2
葡萄 10
洋桔梗 1
柑桔 19
番荔枝 12
鳳梨釋迦 8
茂谷柑 3
臍橙 1
蘭花 3
酪梨 2
花卉 9
白柚 12
椪柑 7
柳橙 7
西施柚 1
蔬果類 2
明尼吉柚 1
葡萄柚 3
葉菜類 8
雜糧 4
雜草 79
葫蘆科 5
茄科 20
芸香科 1
綠肥作物 5
柚類 1
苦瓜 1
扁蒲 1
花椰菜 2
園藝作物 2
芥菜 2
白菜 2
聖誕紅 1
瓜類 10
竹筍 1
蘿蔔 1
多花菊 1
玫瑰 1
唐菖蒲 1
百合 1
甜椒 8
辣椒 3
茄子 3
胡麻 1
咖啡 2
佛手瓜 1
菊科 3
蔥科 4
青江白菜 1
包心白菜 1
包心芥菜 1
芥藍菜 2
甜蜜桃 1
小白菜 1
青花菜 1
花菜類 1
菸草 2
蘇鐵 2
莧菜 1
空心菜 1
紅豆 3
龍葵 1
曼陀羅 1
觀賞植物 1
棉花 1
無患子科 2
洛神葵 1
冬瓜 1


### 文章數量呈現
水稻文章的數量佔了很大比例

In [20]:
import plotly.graph_objects as go
import plotly.express as px

index = 0
count_list = []
name_list = []

for articles in df.articles:
    count_list.append(len(articles))
    name_list.append(df.iloc[index][0])
    index += 1
    
fig_df = pd.DataFrame()
fig_df["作物"] = name_list
fig_df["文章數量"] = count_list

fig = px.bar(fig_df, x="作物", y="文章數量", title="public 作物在文章出現數量")
fig.show()

### 第一層過濾

初步判斷關鍵字的重疊度先形成第一階段的文章組合候選人

In [38]:
# 讀取檔案內容
def get_file_rows(fileNum):
    file = open(f"{currentFolder}/data/{fileNum}.csv", "r")
    rows = file.readlines()
    file.close() 
    return rows

In [37]:
# 加入答案
def add_to_result(item):
    if(item not in test_result):
        test_result.append(item)

In [35]:
# 把 疾病跟害蟲 分開
def get_disease_pest(items):
    disease, pest = [], []
    for item in items:
        if("病" in item):
            disease.append(item)
        else:
            pest.append(item)
    return disease, pest

In [34]:
# 把化學藥劑關鍵字提取出來
def convert_chem(row):
    row = row.replace("\n", "")
    if(row == ""):
        return []
    else:
        return row.split(",")
    return row

In [33]:
# 比較兩篇文章的的化學藥劑相同數
def count_chem_overlap(row1, row2):
    item1 = convert_chem(row1)
    item2 = convert_chem(row2)
    count = 0
    for item in item1:
        if item in item2:
            count += 1
    return count

In [40]:
# read real answer
answer_file = open("TrainLabel.csv", "r")
answer = answer_file.readlines()

ans_dict = {}
total_ans = 0
for i in range(1, len(answer)):
    items = answer[i].strip("\n").split(",")
    test, reference = int(items[0]), int(items[1])
    total_ans += 1
    
    if(test in ans_dict):
        ans_dict[test].append(reference)
    else:
        ans_dict[test] = [reference]     

print("finish:", total_ans)

finish: 1383


In [31]:
def is_related(test1, test2):
    
    rows1 = get_file_rows(test1)
    disease_pest1 = [] if rows1[1] == "\n" else rows1[1].strip("\n").split(",")
    chem1 = len(convert_chem(rows1[2]))

    rows2 = get_file_rows(test2)
    disease_pest2 = rows2[1].strip("\n").split(",")
    chem2 = len(convert_chem(rows2[2]))

    diseases1, pest1 = get_disease_pest(disease_pest1)
    diseases2, pest2 = get_disease_pest(disease_pest2)

    # 判斷疾病主題：如果疾病一樣，則代表有關聯
    result = False
    isDisease = False
    DiseaseCount = 0
    for disease in diseases1:
        if(disease in diseases2):
            isDisease = True
            DiseaseCount += 1

    # 判斷害蟲: 如果疾病+害蟲有重疊的話就算有關聯
    count, isPest = 0, False
    overlap = count_chem_overlap(rows1[2], rows2[2])
    if(isDisease and DiseaseCount >= len(diseases1)-1):
        for bug in pest1:
            if(bug in pest2):  #  and overlap >= chem1/2
                count += 1    
        if(count != 0 and count >= len(pest1)/2 and len(pest1) != 0):
            isPest = True
            result = True
            
        # 沒有害蟲
        elif(count == 0):
            # 為了限縮可能的文章組合，針對 chem 關鍵字類別進行篩選取捨
            if((chem1 != 0 and chem2 != 0) and (overlap >= chem1/2)):
                result = True
            elif(chem1 == 0 and chem2 == 0):
                result = True
    
    # 沒有疾病: 害蟲指示需要相同
    if(len(diseases1) == 0):
        overlapCount = 0
        for bug in pest1:
            if(bug in pest2):
                overlapCount += 1

        if(overlapCount >= len(pest1)/2 and len(pest2) != 0 and overlap >= chem1/2): 
            result = True
        
        # 也沒有害蟲
        if(overlapCount == 0 and len(pest1) == 0):
            result = False

    return result

In [46]:
test_result = []

for index in range(len(df)):
    crop = df.iloc[index][0]
    rice_articles = df.iloc[index]["articles"]
    # 提及同個作物的文章相互比對
    for test1 in rice_articles:
        for test2 in rice_articles:
            
            if(test1 == test2):
                continue
            
            ans = is_related(test1, test2)
            if(ans):
                add_to_result([test1, test2])

print("finish:", len(test_result))

finish: 5918


### 檢查答案
for training data set

In [48]:
# 獲取主要作物名稱
def main_crop(fileNum):
    rows = get_file_rows(fileNum)
    item = rows[0].strip("\n").split(",")[0]
    row2 = [] if rows[1] == "\n" else rows[1].strip("\n").split(",")
    return item

In [51]:
reverse, true, error = 0, 0, 0
trainLabelFile = open("train/trainLabelFile.csv", "w") # record related, unrelated as training data
trainLabelFile.write("Test,Reference,Label\n")

for index in ans_dict:
    articles = ans_dict[index]
    for article in articles:

        if([index, article] in test_result):
            true += 1
            trainLabelFile.write(f"{index}, {article}, 1\n")  # related
        else:
            trainLabelFile.write(f"{index}, {article}, 0\n")  # unrelated
            
            if([article, index] in test_result):
                print(f"[{index}, {article}]({main_crop(index)} {index} -> {main_crop(article)} {article}) reverse exist.")
                reverse += 1
            else:
                print(f"[{index}, {article}] ({main_crop(index)} {index} -> {main_crop(article)} {article}) not in predict answer.")
                trainLabelFile.write(f"{index}, {article}, 0\n")     
                error += 1
trainLabelFile.close()

print("-")
precision = true / len(test_result)
recall = true / (total_ans)
score = 2 * (precision * recall) / (precision + recall)
print(f"precision: {precision}, ({true} / {len(test_result)})")
print(f"recall: {recall}, ({true} / {total_ans})")
print(f"score: {score}")
print(f"reverse: {reverse} | not found: {error}")

[35, 75] (雜草 35 ->  75) not in predict answer.
[35, 191] (雜草 35 ->  191) not in predict answer.
[49, 95] (水稻 49 -> 水稻 95) not in predict answer.
[66, 75] (雜草 66 ->  75) not in predict answer.
[66, 191] (雜草 66 ->  191) not in predict answer.
[73, 67] ( 73 -> 番石榴 67) not in predict answer.
[73, 68] ( 73 -> 番石榴 68) not in predict answer.
[75, 191] ( 75 ->  191) not in predict answer.
[95, 27] (水稻 95 -> 水稻 27) not in predict answer.
[95, 46] (水稻 95 -> 水稻 46) not in predict answer.
[95, 49] (水稻 95 -> 水稻 49) not in predict answer.
[95, 344] (水稻 95 -> 水稻 344) not in predict answer.
[95, 375] (水稻 95 -> 水稻 375) not in predict answer.
[95, 498] (水稻 95 -> 水稻 498) not in predict answer.
[95, 505] (水稻 95 -> 水稻 505) not in predict answer.
[95, 759] (水稻 95 -> 水稻 759) not in predict answer.
[95, 790] (水稻 95 -> 水稻 790) not in predict answer.
[107, 108] (瓜果類 107 ->  108) not in predict answer.
[108, 107] ( 108 -> 瓜果類 107) not in predict answer.
[121, 95] (水稻 121 -> 水稻 95) not in predict answer.
[129, 95

In [63]:
is_related(95, 344) 

False

In [61]:
def display_file(num1, num2):
    directory = f"{currentFolder}/data{currentFolder.capitalize()}Complete/"
    file = open(directory + str(num1) + ".txt", "r")
    print(file.read())
    file.close()
    
    print("- - -")
    
    file = open(directory + str(num2) + ".txt", "r")
    print(file.read())
    file.close()

In [59]:
def get_file(num1, num2):
    rows1 = get_file_rows(num1)
    rows2 = get_file_rows(num2)
    print(rows1)
    print(rows2)

In [57]:
def check(num1, num2):
    display_file(num1, num2)
    get_file(num1, num2)

In [66]:
check(816, 95) 

近日宜蘭防疫所監測縣內水稻田區，發現白葉枯病已現，籲請農民加強防範工作。宜蘭縣政府、防檢局及田邊好幫手關心您。
一、近日因間歇性降雨，且隅有強風，宜蘭防疫所監測本縣員山鄉、五結鄉、冬山鄉、壯圍鄉、礁溪鄉等，部分水稻田區已有輕微白葉枯病發生，請貴管加強宣導農民掌握防治時機及適時防範，減少損失。二、水稻白葉枯病一般都發生在二期稻作，但遭受強風降雨數日，宜蘭一期稻作亦會發生，該病為一種系統性病害，主要危害葉片、葉鞘，偶有危害到穗部，病原菌由傷口或自然開口侵入，常沿葉緣一邊或兩邊，並向中肋擴大，最後到達葉鞘，黃色病斑最後變成白或灰白色；另一種病徵是水浸狀和暗綠色病徵出現不久，罹病葉片即向內捲，葉片上端被害部份凋並成灰白色，露水未乾時，細菌會溢泌在葉緣水孔形成菌泥，隨人員走動或雨水而傳播。高溫環境下，偶而會發生白葉枯病急速萎凋型病徵。
一、勿剪除秧葉尖，以免病原菌由傷口侵入。施用炭化稻殼等含矽資材及減施氮肥以增強葉片強度，減少傷口之發生。二、避免偏用氮素肥料，發病地區儘量避免追施穗肥。三、雨後或晨露末乾前，避免進入稻田，以減少人為傳播。四、幼穗形成期前施用撲殺熱或於發病初期施用克枯爛、鏈四環黴素等，相關藥劑防治可參考防檢局農藥資訊服務網(https://pesticide.baphiq.gov.tw/)。

- - -
發布本(104)年度第二期作水稻白葉枯病發生預報。桃園區農改場、防檢局及田邊好幫手關心您。
一、依據本場研究人員水稻疫情巡迴調查及杜鵑颱風過境影響，部份田區水稻白葉枯病發生已達預報發布標準，為有效控制病害發生，請儘速宣導水稻產銷班及稻農注意防治，確保稻作生產。二、檢附「水稻病蟲害發生預報」及「水稻白葉枯病發生與防治」如附件。
參考植物保護手冊任選一種推薦藥劑加以防治。本病係細菌性病害，主要藉風雨傳播，氣溫25~30℃且遇強風豪雨，易引起葉緣黃白色波浪狀乾枯或葉脈黃白色條紋，嚴重時造成葉片枯捲。防治時應避免過量施用氮肥，晨間露水或下雨後水滴未乾前，避免進入稻田，以減少人為因素傳播病菌。

['水稻\n', '白葉枯病\n', '撲殺熱,克枯爛,鏈四環黴素\n', '宜蘭\n']
['水稻\n', '白葉枯病\n', '\n', '桃園\n']


In [68]:
# record current training data status

file = open(f"{currentFolder}/lstm-train-label.csv", "w")
file.write("Test,Reference\n")

for items in test_result:
    item1, item2 = items[0], items[1]
    label = 0
    if(item1 in ans_dict and item2 in ans_dict[item1]):
        label = 1
    file.write(f"{item1},{item2},{label}\n")

file.close()
print("-")

-


### 寫入初步答案
for public, private data set

In [ ]:
currentFolder = "public"

In [276]:
# record candidates to file

file = open(f"{currentFolder}/all-candidate.csv", "w")
file.write("Test,Reference\n")
for item in test_result:
    file.write(f"{item[0]},{item[1]}\n")
file.close()
print("finish:", len(test_result))

finish: 3257
